In [69]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from collections import defaultdict
import math
import csv

In [5]:
def featureExtractor(filename, features):
    salaries = pd.read_csv(filename)
    teams = salaries.team.unique()
    years = salaries.year.unique()
    allFeatures = []
    for year in years:
        for team in teams:
            result = []
            sliced = salaries[((salaries['team'] == team) & (salaries['year'] == year))]
            teamFeatures = defaultdict(lambda: [])
            for index, row in sliced.iterrows():                
                pos = row['Pos.']
                cap = row['Cap %']
                teamFeatures[pos].append(cap)
            for (position, count) in features:
                posList = sorted(teamFeatures[position], reverse=True)
                for count in range(count):
                    if len(posList) == 0:
                        result.append(0)
                    else:
                        result.append(posList[0])
                        posList = posList[1:]
            allFeatures.append((team, year, result))
    return allFeatures

In [6]:
def distance(vec1, vec2):
    return np.linalg.norm(np.array(vec1)-np.array(vec2))

In [79]:
def kNNTrain(teamFeatures, draft_order, k):
    neigh = KNeighborsClassifier(n_neighbors=k, weights='distance')
    features = []
    output = []
    for (team, year, vec) in teamFeatures:
        if((year, team) in draft_order):
            features.append(vec)
            output.append(draft_order[(year, team)])
    features = np.array(features)
    output = np.array(output)
    output.reshape(1,-1)
    neigh.fit(features, output)
    return neigh 

In [80]:
def kNNPredict(neigh, featureVec):
    return neigh.predict_proba(featureVec)

In [28]:
def get_draft_position(filename):
    '''Returns a dict mapping team to position selected in the first round'''
    teams = {'PIT':'pittsburgh-steelers', 'CIN':'cincinnati-bengals', 'BAL':'baltimore-ravens', 'CLE':'cleveland-browns',
         'NWE':'new-england-patriots', 'BUF':'buffalo-bills', 'MIA':'miami-dolphins', 'NYJ':'new-york-jets',
         'TEN':'tennessee-titans', 'HOU':'houston-texans', 'IND':'indianapolis-colts', 'JAX':'jacksonville-jaguars',
         'KAN':'kansas-city-chiefs', 'OAK':'oakland-raiders', 'DEN':'denver-broncos', 'SDG':'san-diego-chargers',
         'GNB':'green-bay-packers', 'MIN':'minnesota-vikings', 'DET':'detroit-lions', 'CHI':'chicago-bears',
         'DAL':'dallas-cowboys', 'NYG':'new-york-giants', 'PHI':'philadelphia-eagles', 'WAS':'washington-redskins',
         'CAR':'carolina-panthers', 'ATL':'atlanta-falcons', 'NOR':'new-orleans-saints', 'TAM':'tampa-bay-buccaneers',
         'SEA':'seattle-seahawks', 'ARI':'arizona-cardinals', 'STL':'st.-louis-rams', 'SFO':'san-francisco-49ers'}
    draft = pd.read_csv(filename)
#     print draft.head(3)
    draft_order = {}
    for _, row in draft.iterrows():
        if row['Tm'] in teams:
            draft_order[(row['Year'], teams[row['Tm']])] = row['Position Standard']
        else:
            print 'not found', row
    return draft_order
    

In [9]:
def predict_draft_position(feature_vector, draft_picks, teamFeatures):
    kTeams = kNN(feature_vector, teamFeatures, 10)
    similar_drafts = defaultdict(int)
    for team_data in kTeams:
        team, year, distance = team_data
        if (year, team) in draft_picks:
            similar_drafts[draft_picks[(year, team)]] += 1
    return similar_drafts
        

In [70]:
features = [('QB', 1), ('RB', 1), ('DB', 4), ('LB', 3), ('C', 1), ('DE', 2), ('DT', 2), ('G', 2), ('TE', 2), ('WR', 3), ('T', 2)]
teamFeatures = featureExtractor('data/salaries.train.csv', features)
# teamFeatures[0][2] selects the feature vector from the head elem
testFeatures = featureExtractor('data/salaries.test.csv', features)
draftPositions = get_draft_position('data/nfldraft.train.csv')
draftTest = get_draft_position('data/nfldraft.test.csv')
maxCorrect = 0
bestK = 0
for k in range(1, 87):
    picks = 0
    correct = 0
    for i in range(32):
        probabilities = kNN(
            testFeatures[i][2], teamFeatures, draftPositions, k)
#         print testFeatures[i][0], testFeatures[i][1]
        maxProb = 0
        result = None
        for idx, probability in enumerate(probabilities[0]):
            if probability > maxProb:
                maxProb = probability
                result = features[idx]
    #         print features[idx], probability

        if not (testFeatures[i][1], testFeatures[i][0]) in draftTest:
            continue

        if result[0] == draftTest[(testFeatures[i][1], testFeatures[i][0])]:
    #         print '\tCorrect pick', result[0]
            correct += 1
#         else:
    #         print '\tIncorrect pick, guessed', result[0], 'was', draftTest[testFeatures[i][1], testFeatures[i][0]]
        picks += 1
    if correct > maxCorrect:
        bestK = k
        maxCorrect = correct
    print k, correct, '/', picks
print bestK, maxCorrect
    

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

1 6 / 30
2 6 / 30
3 7 / 30
4 4 / 30
5 5 / 30


/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

6 4 / 30
7 3 / 30
8 3 / 30
9 3 / 30


/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

...

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

77 5 / 30
78 5 / 30
79 5 / 30
80 5 / 30


/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

81 5 / 30
82 5 / 30
83 5 / 30
84 5 / 30
85 5 / 30
86 5 / 30
3 7


/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

In [81]:
features = [('QB', 1), ('RB', 1), ('DB', 4), ('LB', 3), ('C', 1), ('DE', 2), ('DT', 2), ('G', 2), ('TE', 2), ('WR', 3), ('T', 2)]
teamFeatures = featureExtractor('data/salaries.train.csv', features)
devFeatures  = featureExtractor('data/salaries.dev.csv', features)
testFeatures = featureExtractor('data/salaries.test.csv', features)

draftTrain = get_draft_position('data/nfldraft.train.csv')
draftDev = get_draft_position('data/nfldraft.dev.csv')
draftTest = get_draft_position('data/nfldraft.test.csv')
picks = 0
correct = 0
for k in range(1, 30):
    data = []
    data.append(['Team'] + [feature[0] for feature in features])
    for i in range(32):
        neigh = kNNTrain(teamFeatures, draftPositions, k)
        probabilities = kNNPredict(
            neigh, testFeatures[i][2])
        maxProb = 0
        result = None
        probabilityDict = defaultdict(float)
        for idx, probability in enumerate(probabilities[0]):
            if probability > maxProb:
                maxProb = probability
                result = features[idx][0]
            probabilityDict[features[idx][0]] = probability

        if not (testFeatures[i][1], testFeatures[i][0]) in draftTest:
            continue

        data.append([testFeatures[i][0]] + list(probabilities[0]) + [draftTest[(testFeatures[i][1], testFeatures[i][0])]])
        if draftTest[(testFeatures[i][1], testFeatures[i][0])] == result:
            correct += 1
        picks += 1
    print data
    with open("output/need.test.csv", "ab+") as f:
        writer = csv.writer(f)
        writer.writerows(data)
        writer.writerows([[k, correct, picks]])

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

[['Team', 'QB', 'RB', 'DB', 'LB', 'C', 'DE', 'DT', 'G', 'TE', 'WR', 'T'], ['pittsburgh-steelers', 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'LB'], ['cincinnati-bengals', 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'T'], ['baltimore-ravens', 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'WR'], ['cleveland-browns', 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'C'], ['new-england-patriots', 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'DT'], ['miami-dolphins', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 'WR'], ['new-york-jets', 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'DE'], ['tennessee-titans', 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'QB'], ['houston-texans', 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'DB'], ['indianapolis-colts', 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 'WR'], ['jacksonville-jaguars', 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 'LB'], ['kansa

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

[['Team', 'QB', 'RB', 'DB', 'LB', 'C', 'DE', 'DT', 'G', 'TE', 'WR', 'T'], ['pittsburgh-steelers', 0.17284685087163029, 0.0, 0.0, 0.0, 0.0, 0.1718153162454388, 0.0, 0.0, 0.65533783288293102, 0.0, 0.0, 'LB'], ['cincinnati-bengals', 0.0, 0.3285084524509379, 0.0, 0.17286369784654673, 0.0, 0.16873427244407577, 0.0, 0.0, 0.32989357725843954, 0.0, 0.0, 'T'], ['baltimore-ravens', 0.0, 0.16353145705264213, 0.1619094981184864, 0.17596488802970567, 0.17282406904977854, 0.0, 0.15673336642981692, 0.0, 0.0, 0.0, 0.1690367213195704, 'WR'], ['cleveland-browns', 0.0, 0.0, 0.36739260037290195, 0.0, 0.14726949090872701, 0.16376430796680153, 0.1506708381631135, 0.0, 0.0, 0.0, 0.17090276258845608, 'C'], ['new-england-patriots', 0.20766888641407868, 0.16649084940601425, 0.0, 0.31639129080383971, 0.0, 0.0, 0.0, 0.0, 0.3094489733760673, 0.0, 0.0, 'DT'], ['miami-dolphins', 0.0, 0.0, 0.31556841750407594, 0.0, 0.16065026037393088, 0.0, 0.35135800982805798, 0.0, 0.17242331229393529, 0.0, 0.0, 'WR'], ['new-york-je

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

...

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

[['Team', 'QB', 'RB', 'DB', 'LB', 'C', 'DE', 'DT', 'G', 'TE', 'WR', 'T'], ['pittsburgh-steelers', 0.054442340934842161, 0.084253963593394834, 0.25887488699111383, 0.12841761969212692, 0.087285235669032027, 0.096431874197487208, 0.0, 0.0, 0.20641467024360505, 0.0, 0.083879408678397965, 'LB'], ['cincinnati-bengals', 0.0, 0.14752242891541925, 0.17762614408549293, 0.13747169453808733, 0.12573269178567123, 0.050955170291487752, 0.0, 0.046537189779409925, 0.22798964402453722, 0.0, 0.086165036579894355, 'T'], ['baltimore-ravens', 0.0, 0.26285125826024752, 0.2661665762644127, 0.053861798054156425, 0.099147099898307292, 0.042760364946413207, 0.092411588116234705, 0.0, 0.089010205854842261, 0.0, 0.093791108605385851, 'WR'], ['cleveland-browns', 0.039440586467729369, 0.082256408831989786, 0.24818150992984139, 0.083971981556027336, 0.08818327138912363, 0.093869264310261061, 0.17851327183362448, 0.0, 0.04276018488170711, 0.041087677736165354, 0.10173584306353059, 'C'], ['new-england-patriots', 0.06

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

[['Team', 'QB', 'RB', 'DB', 'LB', 'C', 'DE', 'DT', 'G', 'TE', 'WR', 'T'], ['pittsburgh-steelers', 0.047278254621034743, 0.10597538414298459, 0.22480945183465884, 0.21030101550057226, 0.075799341574170023, 0.083742370801962077, 0.0, 0.0, 0.17925249299940627, 0.0, 0.072841688525211251, 'LB'], ['cincinnati-bengals', 0.0, 0.16217870612539112, 0.15337199782665178, 0.11870047928177518, 0.14254936282550698, 0.043997443661453695, 0.067761185813789357, 0.040182720885225107, 0.1968585613782193, 0.0, 0.074399542201987412, 'T'], ['baltimore-ravens', 0.0, 0.22621193043681173, 0.26371021439042464, 0.046353901424222771, 0.085326800463711022, 0.071870813358554136, 0.11394806532245667, 0.03525797678925563, 0.076602907013920987, 0.0, 0.080717390800642455, 'WR'], ['cleveland-browns', 0.034120857942569256, 0.071161701485312756, 0.24821721966092633, 0.07264587853361619, 0.076289151492238041, 0.081208220251099583, 0.18839371395453502, 0.0, 0.10440370798862662, 0.035545790283135086, 0.088013758407941192, 'C'

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/usr/lo

In [84]:
def get_draft_probability_matrix():
    features = [('QB', 1), ('RB', 1), ('DB', 4), ('LB', 3), ('C', 1), ('DE', 2), ('DT', 2), ('G', 2), ('TE', 2), ('WR', 3), ('T', 2)]
    teamFeatures = featureExtractor('data/salaries.train.csv', features)
    devFeatures  = featureExtractor('data/salaries.dev.csv', features)
    testFeatures = featureExtractor('data/salaries.test.csv', features)

    draftTrain = get_draft_position('data/nfldraft.train.csv')
    draftDev = get_draft_position('data/nfldraft.dev.csv')
    draftTest = get_draft_position('data/nfldraft.test.csv')

    data = {}
    for i in range(32):
        neigh = kNNTrain(teamFeatures, draftPositions, k)
        probabilities = kNNPredict(
            neigh, testFeatures[i][2])
        probabilityDict = {features[idx][0]: probability for idx, probability in enumerate(probabilities[0])}
        data[testFeatures[i][0]] = list(probabilities[0])
    return data

In [85]:
print get_draft_probability_matrix()

/usr/local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


NameError: free variable 'probabilityDict' referenced before assignment in enclosing scope